# 数据处理

In [8]:
import torchvision
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_set = torchvision.datasets.CIFAR10(root='../Dataset', train=True, transform=torchvision.transforms.ToTensor(), download=True) # 训练集
test_set = torchvision.datasets.CIFAR10(root='../Dataset', train=False, transform=torchvision.transforms.ToTensor(), download=True) # 测试集

train_data_size = len(train_set)
test_data_size = len(test_set)

print(train_data_size, test_data_size)

train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True, drop_last=False)
test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=False, drop_last=False)

Files already downloaded and verified
Files already downloaded and verified
50000 10000


# 构建网络

In [9]:
from torch import nn

class Mynn(nn.Module):

	def __init__(self):
		super().__init__()
		self.sequential = nn.Sequential(
			nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2),
			nn.MaxPool2d(2),
			nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
			nn.MaxPool2d(2),
			nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
			nn.MaxPool2d(2),
			nn.Flatten(), # 将tensor张成一维张量
			nn.Linear(1024, 64),
			nn.Linear(64, 10)
		)
	
	def forward(self, x):
		x = self.sequential(x)
		return x

In [10]:
from torch import optim

mynn = Mynn().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optim = optim.SGD(mynn.parameters(), lr = 1e-2)



In [11]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("logs") # 创建对象 在logs文件夹下保存文件


epochs = 40


for i in range(epochs):
	
	print('-----------第{}轮训练-------------'.format(i + 1))
	
	train_size = 0	
	train_step = 0

	for imgs, targets in train_loader:
		
		imgs = imgs.to(device)
		targets = targets.to(device)

		# 正向推理
		outputs = mynn(imgs)
		loss = loss_fn(outputs, targets)
		
		# 优化
		optim.zero_grad()
		loss.backward()
		optim.step()


		# 观察损失函数	
		train_size += len(imgs)
		train_step += 1

		writer.add_scalar("Train Loss", loss.item(), train_step)
		if train_step % 250 == 0:
			print('{}/{} Loss = {}'.format(train_size, train_data_size, loss.item()))

	
	# 测试
	total_loss = 0

	# 上下文管理器：
	# with 语句是一个上下文管理器，确保在其内部的代码块在启用了 torch.no_grad() 模式下执行。
	# 一旦代码块执行完毕，上下文管理器会恢复之前的状态，如果之前启用了梯度计算，则重新启用。

	# torch.no_grad() 纯推理 不需要记录梯度 节省时间
	with torch.no_grad():
		for imgs, targets in test_loader:

			imgs = imgs.to(device)
			targets = targets.to(device)
			
			outputs = mynn(imgs)
			loss = loss_fn(outputs, targets)
			total_loss += loss.item()

		writer.add_scalar("Test Loss", total_loss/test_data_size, i)
		print('测试集Loss = {}'.format(total_loss/test_data_size))


	if (i + 1) % 10 == 0:
		torch.save(mynn, './model/train_model{}.pth'.format(i))
	
writer.close()	 


-----------第1轮训练-------------
16000/50000 Loss = 2.2679781913757324
32000/50000 Loss = 2.0383412837982178
48000/50000 Loss = 2.015619993209839
测试集Loss = 327.1224995851517
-----------第2轮训练-------------
16000/50000 Loss = 1.8504860401153564
32000/50000 Loss = 1.6888399124145508
48000/50000 Loss = 1.7782565355300903
测试集Loss = 316.9340169429779
-----------第3轮训练-------------
16000/50000 Loss = 2.026477098464966
32000/50000 Loss = 1.6106362342834473
48000/50000 Loss = 1.724058747291565
测试集Loss = 262.1091502904892
-----------第4轮训练-------------
16000/50000 Loss = 1.4752639532089233
32000/50000 Loss = 1.5337363481521606
48000/50000 Loss = 1.4638481140136719
测试集Loss = 248.33262193202972
-----------第5轮训练-------------
16000/50000 Loss = 1.5756809711456299
32000/50000 Loss = 1.4454150199890137
48000/50000 Loss = 1.2594473361968994
测试集Loss = 231.36229169368744
-----------第6轮训练-------------
16000/50000 Loss = 1.6811692714691162
32000/50000 Loss = 1.3971315622329712
48000/50000 Loss = 1.32223284244537